<h1>Capstone Project - The Battle of Neighborhoods</h1>

### Importing libraries required

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


### Downloading and exploring the dataset

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


<b>Converting the data to a pandas dataframe</b>

In [5]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
neighborhoods.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


<b>Use geopy library to get the latitude and longitude values of New York City.</b>

In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


<b>Creating a map of New York with neighborhoods superimposed on top.</b>

In [10]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


In [11]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Foursquare venues

In [12]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [13]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [14]:
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_sushi = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1d2941735')
newyork_venues_sushi.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Planet Tokyo,40.886233,-73.909479,Sushi Restaurant
1,Chinatown,40.715618,-73.994279,Shinsen,40.715608,-73.996611,Japanese Restaurant
2,Chinatown,40.715618,-73.994279,Whole Foods Sushi Bar,40.724000,-73.992277,Sushi Restaurant
3,Chinatown,40.715618,-73.994279,Douzo,40.719069,-73.990781,Japanese Restaurant
4,Chinatown,40.715618,-73.994279,Amano Sushi,40.716017,-73.992818,Sushi Restaurant
5,Chinatown,40.715618,-73.994279,Sushumai Asian Fusion,40.721155,-73.987337,Sushi Restaurant
6,Chinatown,40.715618,-73.994279,Shōji,40.717582,-74.005592,Sushi Restaurant
7,Chinatown,40.715618,-73.994279,Bayard Sushi,40.715614,-73.998960,Sushi Restaurant
8,Chinatown,40.715618,-73.994279,Masa's Raw Bar,40.717082,-74.005260,Sushi Restaurant
9,Chinatown,40.715618,-73.994279,Matsunichi,40.715690,-73.997237,Sushi Restaurant


In [15]:
newyork_venues_sushi.shape

(1089, 7)

In [16]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [17]:
map_newyork_sushi = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_sushi, 'red', map_newyork_sushi)

map_newyork_sushi

In [18]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [19]:
manhattan_grouped = newyork_venues_sushi.groupby('Neighborhood').count()
manhattan_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,21,21,21,21,21,21
Carnegie Hill,20,20,20,20,20,20
Central Harlem,3,3,3,3,3,3
Chelsea,42,42,42,42,42,42
Chinatown,26,26,26,26,26,26
Civic Center,31,31,31,31,31,31
Clinton,38,38,38,38,38,38
East Harlem,3,3,3,3,3,3
East Village,50,50,50,50,50,50


### Analyze each neighborhood

In [20]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_sushi[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_sushi['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant,Noodle House,Poke Place,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,Chinatown,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Chinatown,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [21]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant,Noodle House,Poke Place,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.095238,0.047619,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.809524,0.047619,0.000000
1,Carnegie Hill,0.050000,0.00,0.050000,0.000000,0.00000,0.000000,0.000000,0.100000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.800000,0.000000,0.000000
2,Central Harlem,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
3,Chelsea,0.047619,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.119048,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.809524,0.000000,0.023810
4,Chinatown,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.192308,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.807692,0.000000,0.000000
5,Civic Center,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.096774,0.032258,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.838710,0.032258,0.000000
6,Clinton,0.026316,0.00,0.026316,0.026316,0.00000,0.000000,0.000000,0.105263,0.000000,0.026316,0.00,0.00,0.000000,0.00,0.026316,0.000000,0.000000,0.763158,0.000000,0.000000
7,East Harlem,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
8,East Village,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.120000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.860000,0.000000,0.020000
9,Financial District,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.190476,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.809524,0.000000,0.000000


In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,Theme Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store
1,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Noodle House,Bakery,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
2,Central Harlem,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
3,Chelsea,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
4,Chinatown,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
5,Civic Center,Sushi Restaurant,Japanese Restaurant,Noodle House,Theme Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store
6,Clinton,Sushi Restaurant,Japanese Restaurant,Poke Place,Chinese Restaurant,Cocktail Bar,Asian Restaurant,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant
7,East Harlem,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
8,East Village,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
9,Financial District,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant


Cluster Neighborhoods

In [24]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 2, 0, 0, 0, 1, 2, 0, 0], dtype=int32)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,0,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
5,Manhattan,Manhattanville,40.816934,-73.957385,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
6,Manhattan,Central Harlem,40.815976,-73.943211,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
7,Manhattan,East Harlem,40.792249,-73.944182,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
8,Manhattan,Upper East Side,40.775639,-73.960508,1,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
9,Manhattan,Yorkville,40.775930,-73.947118,0,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Noodle House,Bakery,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
9,Yorkville,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Noodle House,Bakery,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
13,Lincoln Square,Sushi Restaurant,Japanese Restaurant,Smoothie Shop,Chinese Restaurant,Grocery Store,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Cocktail Bar,Deli / Bodega
17,Chelsea,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
19,East Village,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
21,Tribeca,Sushi Restaurant,Noodle House,Japanese Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store
22,Little Italy,Sushi Restaurant,Japanese Restaurant,Noodle House,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
23,Soho,Sushi Restaurant,Japanese Restaurant,Noodle House,Theme Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store
24,West Village,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Sake Bar,Asian Restaurant,Seafood Restaurant,Sandwich Place,Smoothie Shop,Restaurant,Ramen Restaurant
25,Manhattan Valley,Sushi Restaurant,Hawaiian Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store


In [28]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
10,Lenox Hill,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
14,Clinton,Sushi Restaurant,Japanese Restaurant,Poke Place,Chinese Restaurant,Cocktail Bar,Asian Restaurant,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant
15,Midtown,Sushi Restaurant,Asian Restaurant,Seafood Restaurant,Japanese Restaurant,Ramen Restaurant,Bakery,Vegetarian / Vegan Restaurant,Sandwich Place,Sake Bar,Restaurant
16,Murray Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Restaurant,Bakery,Chinese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Sake Bar,Sandwich Place
30,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Noodle House,Bakery,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
33,Midtown South,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Restaurant,Bakery,Chinese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Sake Bar,Sandwich Place
34,Sutton Place,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Steakhouse,Seafood Restaurant,Deli / Bodega,Bakery,Chinese Restaurant,Cocktail Bar,Grocery Store
39,Hudson Yards,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Cocktail Bar,Poke Place,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant


In [29]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
2,Washington Heights,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
3,Inwood,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
4,Hamilton Heights,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
5,Manhattanville,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
6,Central Harlem,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
7,East Harlem,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant


In [30]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upper West Side,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Grocery Store,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Hawaiian Restaurant
20,Lower East Side,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant


In [31]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Roosevelt Island,Sushi Restaurant,Asian Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
18,Greenwich Village,Sushi Restaurant,Japanese Restaurant,Sake Bar,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store
26,Morningside Heights,Sushi Restaurant,Hawaiian Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Japanese Restaurant
35,Turtle Bay,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Steakhouse,Seafood Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store
36,Tudor City,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant
